In [1]:
!pwd

/content


In [2]:
#changing the working directory and entering to dataset directory for my google drive
%cd drive/MyDrive/kaggle/image_classification/

/content/drive/MyDrive/kaggle/image_classification


In [3]:
%cd 10_monkey_species/

/content/drive/MyDrive/kaggle/image_classification/10_monkey_species


In [4]:
!ls

monkey_labels.txt  test  train


In [5]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [6]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [7]:
train_path = 'train'
valid_path = 'test'

In [9]:
# add preprocessing layer to the front of Inception_v3
mobilenet = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

9420800/9406464 [==============================] - 0s 0us/step


In [10]:
# don't train existing weights
for layer in mobilenet.layers:
  layer.trainable = False

In [11]:
# useful for getting number of classes
folders = glob('train/*')

In [12]:
# our layers - you can add more if you want
x = Flatten()(mobilenet.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [13]:
# create a model object
model = Model(inputs=mobilenet.input, outputs=prediction)

In [14]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [15]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [16]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1098 images belonging to 10 classes.


In [18]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 272 images belonging to 10 classes.


In [19]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=6,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/6
35/35 [==============================] - 415s 11s/step - loss: 1.6611 - accuracy: 0.7978 - val_loss: 1.1250 - val_accuracy: 0.8971
Epoch 2/6
35/35 [==============================] - 47s 1s/step - loss: 0.4013 - accuracy: 0.9572 - val_loss: 0.5803 - val_accuracy: 0.9412
Epoch 3/6
35/35 [==============================] - 48s 1s/step - loss: 0.2425 - accuracy: 0.9736 - val_loss: 0.3819 - val_accuracy: 0.9632
Epoch 4/6
35/35 [==============================] - 48s 1s/step - loss: 0.1006 - accuracy: 0.9818 - val_loss: 0.4580 - val_accuracy: 0.9596
Epoch 5/6
35/35 [==============================] - 47s 1s/step - loss: 0.1099 - accuracy: 0.9900 - val_loss: 0.4748 - val_accuracy: 0.9706
Epoch 6/6
35/35 [==============================] - 45s 1s/step - loss: 0.1077 - accuracy: 0.9836 - val_loss: 0.5905 - val_accuracy: 0.9596
